In [1]:
from TimeLLMProcessors import FixedModelTimeLLMProcessor

In [2]:
import pandas as pd

In [9]:
updated_df = pd.read_csv("https://raw.githubusercontent.com/cdcepi/FluSight-forecast-hub/refs/heads/main/target-data/target-hospital-admissions.csv")
updated_df = updated_df[updated_df["location_name"] == "US"]
updated_df = updated_df[["date", "value"]]
updated_df["date"] = pd.to_datetime(updated_df["date"])
updated_df.set_index("date", inplace = True)
updated_df.sort_values(by = "date", inplace = True)

In [ ]:
input_size_df = pd.DataFrame(columns=["Parameter Value", "Forecast 1 MAE", "Forecast 2 MAE", "Forecast 3 MAE", "Forecast 4 MAE", "Forecast 5 MAE"])
patch_len_df = pd.DataFrame(columns=["Parameter Value", "Forecast 1 MAE", "Forecast 2 MAE", "Forecast 3 MAE", "Forecast 4 MAE", "Forecast 5 MAE"])
stride_df = pd.DataFrame(columns=["Parameter Value", "Forecast 1 MAE", "Forecast 2 MAE", "Forecast 3 MAE", "Forecast 4 MAE", "Forecast 5 MAE"])

In [ ]:
#Iteration Loop for input_size

input_sizes = [12, 24, 52, 70]

for size in input_sizes:
    Processor = FixedModelTimeLLMProcessor(overall_df = updated_df, dates = ["2024-10-05", "2024-10-19", "2024-11-02", "2024-11-16", "2024-12-07"])
    Processor.create_training_dfs(value_col="value")
    prompt = "This dataset contains data on weekly flu hospitalizations in the United States. There is a yearly (52 weeks) seasonality"
    Processor.create_fixed_model(h = 4, freq = "W-SAT", model_name = "test_model", prompt = prompt, string_param="input_size", val=size)
    
    Processor.calculate_metrics()
    Processor.create_metrics_df()
    row_df = Processor.metrics_df["MAE"].to_frame().T
    row_df["Parameter Value"] = size
    row_df.rename(columns = {0: "Forecast 1 MAE", 1: "Forecast 2 MAE", 2: "Forecast 3 MAE", 3: "Forecast 4 MAE", 4: "Forecast 5 MAE"}, inplace=True)
    input_size_df = pd.concat([input_size_df, row_df], ignore_index=True)

In [ ]:
input_size_df.set_index("Parameter Value", inplace = True)
input_size_df.style.highlight_min(color='yellow', axis=0)

In [ ]:
#Iteration Loop for patch_len

patch_len_sizes = [2, 4, 8, 16, 32]

for patch_len in patch_len_sizes:
    Processor = FixedModelTimeLLMProcessor(overall_df = updated_df, dates = ["2024-10-05", "2024-10-19", "2024-11-02", "2024-11-16", "2024-12-07"])
    Processor.create_training_dfs(value_col="value")
    prompt = "This dataset contains data on weekly flu hospitalizations in the United States. There is a yearly (52 weeks) seasonality"
    Processor.create_fixed_model(h = 4, freq = "W-SAT", model_name = "test_model", prompt = prompt, string_param="patch_len", val=patch_len)
    
    Processor.calculate_metrics()
    Processor.create_metrics_df()
    row_df = Processor.metrics_df["MAE"].to_frame().T
    row_df["Parameter Value"] = patch_len
    row_df.rename(columns = {0: "Forecast 1 MAE", 1: "Forecast 2 MAE", 2: "Forecast 3 MAE", 3: "Forecast 4 MAE", 4: "Forecast 5 MAE"}, inplace=True)
    patch_len_df = pd.concat([patch_len_df, row_df], ignore_index=True)

In [ ]:
patch_len_df.set_index("Parameter Value", inplace = True)
patch_len_df.style.highlight_min(color='yellow', axis=0)

In [ ]:
#Iteration Loop for stride

strides = [1, 2, 4, 8]

for stride in strides:
    Processor = FixedModelTimeLLMProcessor(overall_df = updated_df, dates = ["2024-10-05", "2024-10-19", "2024-11-02", "2024-11-16", "2024-12-07"])
    Processor.create_training_dfs(value_col="value")
    prompt = "This dataset contains data on weekly flu hospitalizations in the United States. There is a yearly (52 weeks) seasonality"
    Processor.create_fixed_model(h = 4, freq = "W-SAT", model_name = "test_model", prompt = prompt, string_param="stride", val=stride)
    
    Processor.calculate_metrics()
    Processor.create_metrics_df()
    row_df = Processor.metrics_df["MAE"].to_frame().T
    row_df["Parameter Value"] = stride
    row_df.rename(columns = {0: "Forecast 1 MAE", 1: "Forecast 2 MAE", 2: "Forecast 3 MAE", 3: "Forecast 4 MAE", 4: "Forecast 5 MAE"}, inplace=True)
    stride_df = pd.concat([stride_df, row_df], ignore_index=True)

In [ ]:
stride_df.set_index("Parameter Value", inplace = True)
stride_df.style.highlight_min(color='yellow', axis=0)

In [3]:
import optuna

In [4]:
study = optuna.create_study()

[I 2025-06-12 12:13:00,694] A new study created in memory with name: no-name-95586985-9579-41fc-ad87-d2d7b31aa9d5


In [5]:
study?

Type:        Study
String form: <optuna.study.study.Study object at 0x7f709549dc10>
File:        ~/.local/lib/python3.11/site-packages/optuna/study/study.py
Docstring:  
A study corresponds to an optimization task, i.e., a set of trials.

This object provides interfaces to run a new :class:`~optuna.trial.Trial`, access trials'
history, set/get user-defined attributes of the study itself.

Note that the direct use of this constructor is not recommended.
To create and load a study, please refer to the documentation of
:func:`~optuna.study.create_study` and :func:`~optuna.study.load_study` respectively.

In [6]:
study.optimize?

Signature:
study.optimize(
    func: 'ObjectiveFuncType',
    n_trials: 'int | None' = None,
    timeout: 'float | None' = None,
    n_jobs: 'int' = 1,
    catch: 'Iterable[type[Exception]] | type[Exception]' = (),
    callbacks: 'Iterable[Callable[[Study, FrozenTrial], None]] | None' = None,
    gc_after_trial: 'bool' = False,
    show_progress_bar: 'bool' = False,
) -> 'None'
Docstring:
Optimize an objective function.

Optimization is done by choosing a suitable set of hyperparameter values from a given
range. Uses a sampler which implements the task of value suggestion based on a specified
distribution. The sampler is specified in :func:`~optuna.study.create_study` and the
default choice for the sampler is TPE.
See also :class:`~optuna.samplers.TPESampler` for more details on 'TPE'.

Optimization will be stopped when receiving a termination signal such as SIGINT and
SIGTERM. Unlike other signals, a trial is automatically and cleanly failed when receiving
SIGINT (Ctrl+C). If ``n_jobs

In [8]:
study.optimize?

Signature:
study.optimize(
    func: 'ObjectiveFuncType',
    n_trials: 'int | None' = None,
    timeout: 'float | None' = None,
    n_jobs: 'int' = 1,
    catch: 'Iterable[type[Exception]] | type[Exception]' = (),
    callbacks: 'Iterable[Callable[[Study, FrozenTrial], None]] | None' = None,
    gc_after_trial: 'bool' = False,
    show_progress_bar: 'bool' = False,
) -> 'None'
Docstring:
Optimize an objective function.

Optimization is done by choosing a suitable set of hyperparameter values from a given
range. Uses a sampler which implements the task of value suggestion based on a specified
distribution. The sampler is specified in :func:`~optuna.study.create_study` and the
default choice for the sampler is TPE.
See also :class:`~optuna.samplers.TPESampler` for more details on 'TPE'.

Optimization will be stopped when receiving a termination signal such as SIGINT and
SIGTERM. Unlike other signals, a trial is automatically and cleanly failed when receiving
SIGINT (Ctrl+C). If ``n_jobs

In [10]:
updated_df

,value
date,
2022-02-05,1091.0
2022-02-12,1158.0
2022-02-19,1490.0
2022-02-26,1603.0
2022-03-05,1791.0
...,...
2025-05-10,2103.0
2025-05-17,1791.0
2025-05-24,1660.0
